# CWRU Experiments
## Imports

In [1]:
import pandas as pd
import numpy as np

import lib.transformers as tf
from sklearn.pipeline import Pipeline

from lib.model import Ganomaly, GanomalyNet
from lib.visualization import GANomalyBoard, rename_tensorboard_key

from skorch.callbacks import PassthroughScoring
import torch

from torch.utils.tensorboard import SummaryWriter
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import train_test_split

from keras.datasets import mnist
import tensorflow

In [2]:
cwruData0 = pd.read_parquet('data/cwru0.parquet')
cwruData1 = pd.read_parquet('data/cwru1.parquet')

cwruData = pd.concat([cwruData0, cwruData1])

## Settings

In [3]:
torch.manual_seed(0)
device = torch.device("cuda:0") if torch.cuda.is_available() else 'cpu'

## Initializing Model(s)

In [4]:
nz = 100  # size of the latent z vector
ngf = 64  # units of generator
ndf = 64  # units of discriminator
nc = 1  # number of channels
batch_size = 32
lr = 0.0002
beta1 = 0.5  # for adam
max_epochs = 5
ngpu = 0
isize = 32  # 32 is easier than 28 to work with
workers = 2  # for dataloader

In [5]:
ganomaly = GanomalyNet(
    Ganomaly,
    module__isize = isize,
    module__nz=nz,
    module__ndf=ndf,
    module__ngf=ngf,
    module__nc=nc,
    module__ngpu=ngpu,
    
    module__w_lat = 1,
    
    criterion=torch.nn.BCELoss,

    optimizer_gen=torch.optim.Adam,
    optimizer_gen__lr=0.0002,
    optimizer_gen__betas=(beta1, 0.999),

    optimizer_dis=torch.optim.Adam,
    optimizer_dis__lr=0.00002,
    optimizer_dis__betas=(beta1, 0.999),

    batch_size=batch_size,
    max_epochs=100,

    train_split=False,  # not implemented
    iterator_train__shuffle=True,
    iterator_train__num_workers=workers,
    iterator_valid__num_workers=workers,

    callbacks=[
        PassthroughScoring('loss_dis', on_train=True),
        PassthroughScoring('loss_gen', on_train=True),
        PassthroughScoring('loss_gen_fra', on_train=True),
        PassthroughScoring('loss_gen_app', on_train=True),
        PassthroughScoring('loss_gen_lat', on_train=True)  
    ]
)

## Data Preparation
Building a pipeline of custom transformers to fetch and preprocess CWRU data.

In [6]:
image_size = 56

selection_pipeline = \
Pipeline(steps=[
                ('DataSelector', tf.DataSelector(columns = ['fanEndData', 'driveEndData'], column_values = {'condition': ['Normal Baseline'], 'sampleRate': [12000]})),
                ('ArrayFlattener', tf.ArrayFlattener()),
                #('ArrayEqualizer', tf.ArrayEqualizer()),
                ('ArrayChunker', tf.ArrayChunker(image_size**2)),
                ('ArrayFlattener2', tf.ArrayFlattener()),
                ('ArrayReshaper', tf.ArrayReshaper((1, image_size, image_size)))
               ])

chunked_normal_data = selection_pipeline.transform(cwruData)

C:\Users\jlsachse\Documents\Bachelorarbeit\Implementation\ganomaly\lib\transformers.py:87: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  X_ = np.array(X_)


Train-test-split of the normal CWRU data.

In [7]:
train, test = train_test_split(chunked_normal_data, train_size = 400, test_size= 451, random_state = 0)

In [8]:
len(chunked_normal_data)

1078

## GridSearchCV

In [9]:
search_parameters = {
    'module_w_fra': list(range(0, 101, 10)),
    'module_w_app': list(range(0, 101, 10)),
    'module_w_lat': list(range(0, 101, 10)),
}

In [10]:
ganomaly_gs = GridSearchCV(ganomaly, search_parameters, refit=False, cv=4)

In [11]:
#ganomaly_gs.fit(train)

## Training Visualization (best parameters)
Adding a TensorBoard for the visualization of the training.

In [12]:
ganomaly_board = GANomalyBoard(SummaryWriter(), key_mapper = rename_tensorboard_key, close_after_train = False)
ganomaly.callbacks += [ganomaly_board]

ganomaly.callbacks

In [13]:
ganomaly.fit(train)

  epoch    loss_dis    loss_gen    loss_gen_app    loss_gen_fra    loss_gen_lat    train_loss      dur
-------  ----------  ----------  --------------  --------------  --------------  ------------  -------
      1      0.6467      1.7578          0.1721          0.7455          0.8402        2.4046  19.3367


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x000001CB14012168>
Traceback (most recent call last):
  File "C:\Users\jlsachse\Anaconda3\envs\ganomaly\lib\site-packages\torch\utils\data\dataloader.py", line 1203, in __del__
    self._shutdown_workers()
  File "C:\Users\jlsachse\Anaconda3\envs\ganomaly\lib\site-packages\torch\utils\data\dataloader.py", line 1177, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "C:\Users\jlsachse\Anaconda3\envs\ganomaly\lib\multiprocessing\process.py", line 140, in join
    res = self._popen.wait(timeout)
  File "C:\Users\jlsachse\Anaconda3\envs\ganomaly\lib\multiprocessing\popen_spawn_win32.py", line 104, in wait
    res = _winapi.WaitForSingleObject(int(self._handle), msecs)
KeyboardInterrupt: 


<class 'lib.model.GanomalyNet'>[initialized](
  module_=Ganomaly(
    (l_fra): BCELoss()
    (l_app): L1Loss()
    (l_dis): L1Loss()
    (discriminator): NetD(
      (features): Sequential(
        (0): Conv2d(1, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
        (1): LeakyReLU(negative_slope=0.2, inplace=True)
        (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
        (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (4): LeakyReLU(negative_slope=0.2, inplace=True)
        (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
        (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (7): LeakyReLU(negative_slope=0.2, inplace=True)
      )
      (classifier): Sequential(
        (0): Conv2d(256, 100, kernel_size=(4, 4), stride=(1, 1), bias=False)
        (Sigmoid): Sigmoid()
      )
    )
    (generator):

In [14]:
ganomaly.predict(test).mean()

KeyboardInterrupt: 

In [ ]:
ganomaly.predict(selection_pipeline.set_params(DataSelector__column_values = {'condition': ['Outer Race Fault'], 'sampleRate': [12000]}).transform(cwruData)[:200])